In [1]:
from functools import partial 

from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb
from sepal_ui.scripts import utils as su
import ipyvuetify as v

from scripts import gee_process as gee_run 
from utils import messages as ms
from utils import parameters as pm

In [2]:
#class io as mutable object 
class Ga_gee_io:
    def __init__(self):
        #input 
        self.year = None
        
ga_gee_io = Ga_gee_io()

In [3]:
#output 
ga_gee_output = wf.OutputWidget(ms.GEE_INTRO)

#btn 
ga_gee_btn = wf.ProcessBtn(ms.GEE_BTN)

#input 
ga_year = v.Select(items=pm.getAvailableYear(), label=ms.GEE_SELECT, v_model=None)

#bind the inputs
wb.bind(ga_year, ga_gee_io, 'year', ga_gee_output)

#tile
id_ = 'gee_widget'
title = 'Retrieve alerts'

ga_gee = wf.Tile(
    id_,
    title,
    btn=ga_gee_btn,
    output=ga_gee_output,
    inputs=[ga_year]
)

In [4]:
#create a custom function for the process
def process_start(widget, event, data, output):
    
    asset = getattr(ga_aoi_io, 'assetId')
    year = getattr(ga_gee_io, 'year')
    
    #toggle the loading button
    su.toggleLoading(widget)
    
    su.displayIO(output, asset)
    
    #check inputs
    if not wb.checkInput(asset, output, ms.NO_AOI): return su.toggleLoading(widget)
    if not wb.checkInput(year, output, ms.NO_YEAR): return su.toggleLoading(widget)
    
    try:
        gee_run.gee_process(asset, year, output) 
    except Exception as e: 
        su.displayIO(output, str(e), 'error')
    
    #toggle the loading button
    su.toggleLoading(widget)
    
    return 

ga_gee_btn.on_event('click', partial(
    process_start,
    output=ga_gee_output,
))
    

In [5]:
ga_gee

Layout(align_center=True, children=[Card(children=[Html(children=['Retrieve alerts'], tag='h2'), Flex(children…